# Solving the Pick-and-Place Environment in Robosuite

## Abstract

## Introduction




## Environment

Blah Blah about robosuite and Stable Baselines


## Implementation

1. Import numpy and robosuite



In [21]:
import numpy as np
import os
import robosuite as suite


from robosuite import load_controller_config
from robosuite.environments.base import register_env
from robosuite.controllers import load_controller_config
from stable_baselines3.common.save_util import save_to_zip_file, load_from_zip_file
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from robosuite.wrappers import GymWrapper

from stable_baselines3 import PPO



2. Creating Standardized Environment

In [22]:
# create environment instance
controller_config = load_controller_config(default_controller="OSC_POSE")


def reward(self, action=None):
    reward = 1
    return reward

def make_robosuite_env(env_id, options, rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param options: (dict) additional arguments to pass to the specific environment class initializer
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = GymWrapper(suite.make(env_id, **options))
        env = Monitor(env)
        env.render_mode = 'mujoco'
        return env
    set_random_seed(seed)
    return _init


3. Simulate steps

In [23]:
env_options = {"robots":"IIWA",
    "controller_configs":controller_config,
    "has_renderer":True,
    "has_offscreen_renderer":False,
    "use_camera_obs":False,
    "reward_shaping":True,
    }

continue_training_model_filename = "model"
#continue_training_model_filename = None


env = SubprocVecEnv([make_robosuite_env("PickPlace", env_options, i, 0) for i in range(3)])

# Create callback
checkpoint_callback = CheckpointCallback(save_freq=1000000, save_path='./checkpoints/', 
                        name_prefix="model", verbose=2)
    
        # Train new model
if continue_training_model_filename is None:

    # Normalize environment
    env = VecNormalize(env)
    #env.render_mode = "mujoco"

    # Create model
    model = PPO("MlpPolicy", env, verbose=1)

    print("Created a new model")

# Continual training
else:

    # Join file paths
    continue_training_model_path = os.path.join(".", continue_training_model_filename + '.zip')
    continue_training_vecnormalize_path = os.path.join(".", 'vec_normalize_' + continue_training_model_filename + '.pkl')

    print(f"Continual training on model located at {continue_training_model_path}")

    # Load normalized env 
    env = VecNormalize.load(continue_training_vecnormalize_path, env)

    # Load model
    model = PPO.load(continue_training_model_path, env=env)


for i in range(0):
    print("Starting learning iteration " + str(i))
    model.learn(total_timesteps=100000)
    model.save('model.zip')
    print("Saved learing iteration " + str(i) + " successfully")
    env.save('vec_normalize_model.pkl')

vec_env = model.get_env()
vec_env.render_mode = 'mujoco'
obs = vec_env.reset()
for i in range(10000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render_mode = 'mujoco'
    vec_env.render()

vec_env.close
env.close

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/tobi/Documents/IKfIR/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)
[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/tobi/Documents/IKfIR/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)
[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/tobi/Documents/IKfIR/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)
Process ForkServerProcess-21:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", li

ConnectionResetError: [Errno 104] Connection reset by peer